In [ ]:
import numpy as np
from qiskit import QuantumCircuit, transpile
from qiskit_aer import QasmSimulator
from scipy.spatial.distance import cosine

# 설정
n_qubits = 3
shots = 1024
n_trials = 30
simulator = QasmSimulator()
base_theta = np.array([0.25, 0.5, 0.75])

# 회로 생성
def make_circuit(theta):
    qc = QuantumCircuit(n_qubits, n_qubits)
    for i, angle in enumerate(theta):
        qc.ry(angle * np.pi, i)
    qc.measure(range(n_qubits), range(n_qubits))
    return qc

# 벡터 변환
def counts_to_vector(counts, n=n_qubits):
    total = sum(counts.values())
    vec = np.zeros(n)
    for bitstring, count in counts.items():
        prob = count / total
        bits = [int(b) for b in bitstring[::-1]]
        vec += np.array(bits) * prob
    return vec

# 유사도
def similarity(v1, v2):
    return 1 - cosine(v1, v2)

# 실행 함수
def run(theta):
    qc = make_circuit(theta)
    tqc = transpile(qc, simulator)
    result = simulator.run(tqc, shots=shots).result()
    return result.get_counts()

# 기준 벡터
counts_C = run(base_theta)
vec_C = counts_to_vector(counts_C)

# 반복 실행
sim_A_list = []
sim_B_list = []

for _ in range(n_trials):
    # A: 구조 반복
    counts_A = run(base_theta)
    vec_A = counts_to_vector(counts_A)
    sim_A_list.append(similarity(vec_C, vec_A))

    # B: 구조 제거
    shuffled_theta = np.random.permutation(base_theta)
    counts_B = run(shuffled_theta)
    vec_B = counts_to_vector(counts_B)
    sim_B_list.append(similarity(vec_C, vec_B))

# 결과 출력
print("🔁 조건 A (구조 반복) 유사도")
print(f"  평균: {np.mean(sim_A_list):.4f}")
print(f"  표준편차: {np.std(sim_A_list):.4f}\n")

print("🔀 조건 B (무작위 구조) 유사도")
print(f"  평균: {np.mean(sim_B_list):.4f}")
print(f"  표준편차: {np.std(sim_B_list):.4f}")


In [ ]:
#🔁 조건 A (구조 반복) 유사도
  #평균: 0.9996
  #표준편차: 0.0003

#🔀 조건 B (무작위 구조) 유사도
  #평균: 0.7391
  #표준편차: 0.1728